# Import Libraries and CSV

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import re
import nltk

In [2]:
df = pd.read_csv('alldata.csv')

In [3]:
new = df["location"].str.split(", ", n=1,expand = True)
df['city'] = new[0]
df['state'] = new[1].str.split(" ",n=1,expand = True)[0]
df = df.drop(['location', 'reviews', 'company'], axis=1)
df = df.drop_duplicates()
df = df.dropna()
df.reset_index(drop=True, inplace=True)

In [4]:
df.head()

,position,description,city,state
0,Development Director,Development Director\nALS Therapy Development ...,Atlanta,GA
1,An Ostentatiously-Excitable Principal Research...,"Job Description\n\n""The road that leads to acc...",Atlanta,GA
2,Data Scientist,"Growing company located in the Atlanta, GA are...",Atlanta,GA
3,Data Analyst,DEPARTMENT: Program OperationsPOSITION LOCATIO...,Atlanta,GA
4,Assistant Professor -TT - Signal Processing & ...,DESCRIPTION\nThe Emory University Department o...,Atlanta,GA


In [5]:
# Number of overlapped position names
lower_position = [position.lower() for position in df['position']]
n = 0
for position in lower_position:
    if 'scien' and 'engineer' in position or 'scien' and 'anal' in position or 'engineer' and 'anal' in position:
        n += 1
print(n)

2515


# Position

## Checking Values

In [6]:
## cleaning Position values
df['position'] = df['position'].apply(lambda x: re.sub(r'[^\w\s]','', x))
df['position'] = df['position'].apply(lambda x: re.sub(r'\r\n',' ', x))
df['position'] = df['position'].apply(lambda x: re.sub(r'\s+', ' ', x))

## putting all the Position words in the same list
position_check = ""
for job in df['position']:
    position_check += " " + job

## tokenizing
from nltk.tokenize import word_tokenize
all_position_low = position_check.lower()
position_tokens = word_tokenize(str(all_position_low))

df.head()

,position,description,city,state
0,Development Director,Development Director\nALS Therapy Development ...,Atlanta,GA
1,An OstentatiouslyExcitable Principal Research ...,"Job Description\n\n""The road that leads to acc...",Atlanta,GA
2,Data Scientist,"Growing company located in the Atlanta, GA are...",Atlanta,GA
3,Data Analyst,DEPARTMENT: Program OperationsPOSITION LOCATIO...,Atlanta,GA
4,Assistant Professor TT Signal Processing Machi...,DESCRIPTION\nThe Emory University Department o...,Atlanta,GA


## Standardising Position Names

In [7]:
lower_position = [position.lower() for position in df['position']]
category = []
for position in lower_position:
    if "anal" in position or "visua" in position:
        category.append('data analyst')
    elif "engineer" in position or "artificial intelligence" in position or "ai" in position or "software" in position:
        category.append('data engineer')
    elif "data scien" in position or "machine learning" in position or "scien" in position:
        category.append('data scientist')
    else:
        category.append('other')
df['position'] = category

In [8]:
df['position'].value_counts().head(20).sort_values(ascending = True)

data analyst      1236
data engineer     1509
other             1660
data scientist    2503
Name: position, dtype: int64

# Description

## Checking Values

In [9]:
## cleaning description values
df['description'] = df['description'].apply(lambda x: re.sub(r'[^\w\s]','', x))
df['description'] = df['description'].apply(lambda x: re.sub(r'\r\n',' ', x))
df['description'] = df['description'].apply(lambda x: re.sub(r'\s+', ' ', x))

## Tokenize the Words
- using tokenize to count the words inside description

In [10]:
from nltk.tokenize import word_tokenize
descriptions = []
for desc in df['description']:
    desc = desc.lower()
    desc = word_tokenize(str(desc))
    descriptions.append(desc)

df['description'] = descriptions

## Frequency of Positions, Tools, Skills, Degree, Major

In [11]:
# hard skills only
matrix = {'data analyst': {},
         'data engineer': {},
         'data scientist': {},
         'other': {}}
positions = ['data analyst', 'data engineer', 'data scientist', 'other']
tool_list = ['python','sql', 'mysql', 'excel', 'tableau']
skill_list = ['machine learning','data analysis','visualization',
            'modeling','statistical analysis','research','deep learning',
            'optimization','decision tree','logistic','random forest',
            'ab testing','web scraping','neural network',
            'communication skills','artificial intelligence']
degree_list = ['master',"masters",'phd','bachelor',"bachelors",
             'high school','No degree']
major_list = ['computer science', 'engineering',
            'mathematics','statistics','economics','business','marketing',
             'psychology', 'health', 'geography']
var_list_of_lists = [tool_list, skill_list, degree_list, major_list]

for pos in positions:
        for var_list in var_list_of_lists:
            for var in var_list:
                n = 0
                for i in list(df.loc[df['position'] == pos].index):
                    p = df.iloc[i,1]
                    if len(var) == 1:
                        if var in p:
                            n += 1
                    else:
                        if var in ' '.join(str(e) for e in p):
                            n += 1
                matrix[pos].update({var: n})

epic_df = pd.DataFrame.from_dict(matrix, orient='columns')

In [12]:
epic_df.head(50)

,data analyst,data engineer,data scientist,other
python,429,817,1334,239
sql,475,454,789,190
mysql,31,76,45,19
excel,756,646,1169,973
tableau,208,47,186,49
machine learning,210,708,1291,205
data analysis,308,174,697,238
visualization,257,159,479,113
modeling,304,312,824,182
statistical analysis,137,70,300,86


In [13]:
epic_df.to_csv('epic_df.csv')